In [58]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()


data = pd.read_csv("data/news.csv")
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


## Data Preprocessing

In [59]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [60]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])


In [61]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1


### Tokenization

In [62]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])


In [63]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1) + 1
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
    sequences1,  padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

### Generating Word Embedding

In [64]:
embeddings_index = {}
with open('glove.6B/glove.6B.50d.txt', encoding='utf8') as f:  # Fixed encoding
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1 + 1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector


## Creating Model Architecture

In [65]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
                              input_length=max_length, weights=[
                                  embeddings_matrix],
                              trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 54, 50)            377650    
                                                                 
 dropout_7 (Dropout)         (None, 54, 50)            0         
                                                                 
 conv1d_7 (Conv1D)           (None, 50, 64)            16064     
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 12, 64)            0         
 g1D)                                                            
                                                                 
 lstm_7 (LSTM)               (None, 64)                33024     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                      

In [66]:
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,
                    epochs=num_epochs,
                    validation_data=(testing_padded,
                                     testing_labels),
                    verbose=2)


Train on 2700 samples, validate on 300 samples
Epoch 1/50


c:\Users\Mayank Jaiswal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 2s - loss: 0.6464 - accuracy: 0.6252 - val_loss: 0.5579 - val_accuracy: 0.6933 - 2s/epoch - 792us/sample
Epoch 2/50
2700/2700 - 1s - loss: 0.5808 - accuracy: 0.6967 - val_loss: 0.5197 - val_accuracy: 0.7200 - 598ms/epoch - 221us/sample
Epoch 3/50
2700/2700 - 1s - loss: 0.5319 - accuracy: 0.7333 - val_loss: 0.5009 - val_accuracy: 0.7400 - 609ms/epoch - 226us/sample
Epoch 4/50
2700/2700 - 1s - loss: 0.4939 - accuracy: 0.7641 - val_loss: 0.5032 - val_accuracy: 0.7467 - 633ms/epoch - 234us/sample
Epoch 5/50
2700/2700 - 1s - loss: 0.4420 - accuracy: 0.7870 - val_loss: 0.4864 - val_accuracy: 0.7267 - 641ms/epoch - 238us/sample
Epoch 6/50
2700/2700 - 1s - loss: 0.4093 - accuracy: 0.8089 - val_loss: 0.5006 - val_accuracy: 0.7467 - 633ms/epoch - 234us/sample
Epoch 7/50
2700/2700 - 1s - loss: 0.3603 - accuracy: 0.8419 - val_loss: 0.4808 - val_accuracy: 0.7867 - 705ms/epoch - 261us/sample
Epoch 8/50
2700/2700 - 1s - loss: 0.3419 - accuracy: 0.8504 - val_loss: 0.4704 - val_accuracy: 0.

In [71]:
# sample text to check if fake or not
X = "Will the Media Reset After the Election or Are We Stuck With This Tabloid Stuff?"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
                          padding=padding_type,
                          truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
    print("This news is True")
else:
    print("This news is false")


This news is false
